In [2]:
import pandas as pd
import random
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [3]:
region_df = pd.read_csv('city_region_mapping.csv')
df = pd.read_csv('SampleDataFoodSales.csv')
unspsc_df = pd.read_csv('unspsc_data.csv')
# unspsc_df['Price'] = pd.Series([round(random.uniform(10, 500), 2) for _ in range(len(unspsc_df))])

In [4]:
# # Mapping the ID from Table 1 to each row in Table 2
# process_region= pd.merge(region_df.assign(key=1), df[['ID']].assign(key=1), on='key').drop('key', axis=1)
# process_region = process_region[['ID', 'State ', 'City', 'Region']]

In [5]:
reg = region_df.head()
reg= pd.merge(reg.assign(key=1), df[['ID']].assign(key=1), on='key').drop('key', axis=1)
reg = reg.iloc[[891]]
# reg['RID'] = 'REG_05'
# reg = reg[['ID','RID','Region','City']]
row = {'ID': 'ID07351', 'Region': 'East', 'City': 'Boston'}
reg = reg.append(row, ignore_index=True)
reg = reg.reset_index(drop = True)

C:\Users\shashi.kiran\AppData\Local\Temp\ipykernel_38844\1520189931.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reg = reg.append(row, ignore_index=True)


In [6]:
reg

,State,City,Region,ID
0,Texas,Houston,South,ID07510
1,NaN,Boston,East,ID07351


In [7]:
df_new = df.copy()

# mapping = {'Boston': 'REG_01',
#     'New York': 'REG_02',
#     'Los Angeles': 'REG_03',
#     'San Diego': 'REG_04'}
# df_new['RID'] = df_new['City'].map(mapping)

In [8]:
df_new

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
0,ID07351,1-Jan,East,Boston,Bars,Carrot,33,1.77,58.41
1,ID07352,4-Jan,East,Boston,Crackers,Whole Wheat,87,3.49,303.63
2,ID07353,7-Jan,West,Los Angeles,Cookies,Chocolate Chip,58,1.87,108.46
3,ID07354,10-Jan,East,New York,Cookies,Chocolate Chip,82,1.87,153.34
4,ID07355,13-Jan,East,Boston,Cookies,Arrowroot,38,2.18,82.84
...,...,...,...,...,...,...,...,...,...
239,ID07590,18-Dec,East,Boston,Cookies,Arrowroot,34,2.18,74.12
240,ID07591,21-Dec,East,Boston,Cookies,Chocolate Chip,245,1.87,458.15
241,ID07592,24-Dec,East,Boston,Crackers,Whole Wheat,30,3.49,104.70
242,ID07593,27-Dec,West,Los Angeles,Bars,Bran,30,1.87,56.10


### Multi Table Approach -- WIP

In [46]:
from sdv.metadata import MultiTableMetadata

metadata = MultiTableMetadata()

In [47]:
multi_table_data = {
    'product_table': df_new,
    'region': reg,
}

In [48]:
metadata.detect_table_from_dataframe(
    table_name='product_table',
    data=df_new
)

metadata.detect_table_from_dataframe(
    table_name='region',
    data=reg
)

In [49]:
metadata.update_column(
    table_name='product_table',
    column_name='ID',
    sdtype='id',
    regex_format='ID[0-9]{5}')

metadata.update_column(
    table_name='region',
    column_name='ID',
    sdtype='id',
    regex_format='ID[0-9]{5}')

metadata.update_column(
    table_name='region',
    column_name='City',
    sdtype='city',
    pii=True
)

# metadata.update_column(
#     table_name='product_table',
#     column_name='RID',
#     sdtype='id',
#     regex_format='RID_[0-9]{2}')

# metadata.update_column(
#     table_name='region',
#     column_name='RID',
#     sdtype='id',
#     regex_format='RID_[0-9]{2}')

In [50]:
metadata.set_primary_key(
    table_name='product_table',
    column_name='ID'
)

# metadata.set_primary_key(
#     table_name='region',
#     column_name='City'
# )

# # metadata.add_alternate_keys(
# #     table_name='product_table',
# #     column_names=['RID']
# # )

# metadata.add_alternate_keys(
#     table_name='region',
#     column_names=['ID']
# )

In [51]:
metadata.add_relationship(
    parent_table_name='product_table',
    child_table_name='region',
    parent_primary_key='ID',
    child_foreign_key='ID'
)

In [52]:
metadata.validate()

In [53]:
metadata

{
    "tables": {
        "product_table": {
            "columns": {
                "ID": {
                    "sdtype": "id",
                    "regex_format": "ID[0-9]{5}"
                },
                "Date": {
                    "sdtype": "categorical"
                },
                "Region": {
                    "sdtype": "categorical"
                },
                "City": {
                    "sdtype": "categorical"
                },
                "Category": {
                    "sdtype": "categorical"
                },
                "Product": {
                    "sdtype": "categorical"
                },
                "Qty": {
                    "sdtype": "numerical"
                },
                "UnitPrice": {
                    "sdtype": "numerical"
                },
                "TotalPrice": {
                    "sdtype": "numerical"
                }
            },
            "primary_key": "ID"
        },
        "region": {


In [54]:
# synthesizer.update_transformers(
#     table_name='product_table',
#     column_name_to_transformer={'ID' :RegexGenerator(regex_format='ID[0-9]{5}', enforce_uniqueness=True)}
#         )

# synthesizer.update_transformers(
#     table_name='product_table',
#     column_name_to_transformer={'RID' :RegexGenerator(regex_format='RID_[0-9]{2}', enforce_uniqueness=True)}
#         )

# synthesizer.update_transformers(
#     table_name='region',
#     column_name_to_transformer={'ID' :RegexGenerator(regex_format='ID[0-9]{5}', enforce_uniqueness=True)}
#         )

# synthesizer.update_transformers(
#     table_name='region',
#     column_name_to_transformer={'RID' :RegexGenerator(regex_format='RID_[0-9]{2}', enforce_uniqueness=True)}
#         )

In [55]:
from sdv.multi_table import HMASynthesizer

# Step 1: Create the synthesizer
synthesizer = HMASynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(multi_table_data)

# Step 3: Generate synthetic data
synthetic_data = synthesizer.sample()

In [56]:
synthetic_data['product_table']['City'].value_counts()

Boston         79
New York       61
Los Angeles    53
San Diego      51
Name: City, dtype: int64

### CTGAN Same Results results as Version 1

In [ ]:
from ctgan import CTGAN

In [ ]:
discrete_col = ['ID', 'Date', 'Region', 'City', 'Category', 'Product']

In [ ]:
ctgan = CTGAN(epochs = 10)
ctgan.fit(df, discrete_col)
synt_data = ctgan.sample(1000)

In [ ]:
synt_data.City.value_counts()

### Single Table Approach with external data append in the likes of UNSPSC & Region Mapping

In [57]:
df.Category.value_counts()

Cookies     95
Bars        94
Snacks      29
Crackers    26
Name: Category, dtype: int64

In [58]:
#UNSPSC Data prep and filterring
selected_family = ['Beverages', 'Prepared and preserved foods', 'Bread and bakery products']

filterd_unspsc = unspsc_df[unspsc_df['Family Name'].isin(selected_family)]
filterd_unspsc = filterd_unspsc.rename(columns = {'Family Name': 'Category', 'Commodity Name': 'Product'})
filterd_unspsc = filterd_unspsc[['Category', 'Product']]
filterd_unspsc = filterd_unspsc.reset_index(drop = True)
filterd_unspsc['ID'] = ['ID' + str(i).zfill(5) for i in range(1, len(filterd_unspsc)+1)]
filterd_unspsc = filterd_unspsc[['ID', 'Category']]

In [59]:
#Region Data prep and filtering
region_df = region_df[['Region','City']]
region_df['ID'] = ['ID' + str(i).zfill(5) for i in range(200, 200 + len(region_df))]
region_df = region_df[['ID','Region','City']]

In [60]:
df_final = df.append(filterd_unspsc, ignore_index =True)
df_final = df_final.append(region_df, ignore_index =True)

C:\Users\shashi.kiran\AppData\Local\Temp\ipykernel_38844\3571460753.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df.append(filterd_unspsc, ignore_index =True)
C:\Users\shashi.kiran\AppData\Local\Temp\ipykernel_38844\3571460753.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(region_df, ignore_index =True)


In [61]:
#adding some random values to the dataset as MRP Data not available
df_final_mean = df_final.fillna(df_final.mean()).round(0)

df_final_mean['Date'] = df_final_mean['Date'].fillna(df_final_mean['Date'].mode()[0])
df_final_mean['Region'] = df_final_mean['Region'].fillna(df_final_mean['Region'].mode()[0])
df_final_mean['City'] = df_final_mean['City'].fillna(df_final_mean['City'].mode()[0])
df_final_mean['Category'] = df_final_mean['Category'].fillna(df_final_mean['Category'].mode()[0])
df_final_mean['Product'] = df_final_mean['Product'].fillna(df_final_mean['Product'].mode()[0])

C:\Users\shashi.kiran\AppData\Local\Temp\ipykernel_38844\34240211.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_final_mean = df_final.fillna(df_final.mean()).round(0)


In [62]:
df_final_mean

,ID,Date,Region,City,Category,Product,Qty,UnitPrice,TotalPrice
0,ID07351,1-Jan,East,Boston,Bars,Carrot,33.0,2.0,58.0
1,ID07352,4-Jan,East,Boston,Crackers,Whole Wheat,87.0,3.0,304.0
2,ID07353,7-Jan,West,Los Angeles,Cookies,Chocolate Chip,58.0,2.0,108.0
3,ID07354,10-Jan,East,New York,Cookies,Chocolate Chip,82.0,2.0,153.0
4,ID07355,13-Jan,East,Boston,Cookies,Arrowroot,38.0,2.0,83.0
...,...,...,...,...,...,...,...,...,...
735,ID00526,1-Apr,Northeast,Lynn,Beverages,Carrot,63.0,2.0,137.0
736,ID00527,1-Apr,Midwest,Carmel,Beverages,Carrot,63.0,2.0,137.0
737,ID00528,1-Apr,West,Longmont,Beverages,Carrot,63.0,2.0,137.0
738,ID00529,1-Apr,South,Tuscaloosa,Beverages,Carrot,63.0,2.0,137.0


In [63]:
# df_final_mean.to_csv('processed_with_append.csv')

### Meta Data with Constraint Creation

In [64]:
from sdv.metadata import SingleTableMetadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_final)
#metadata.set_primary_key(column_name='ID')
constraint = {
    'constraint_class': 'FixedCombinations',
    'constraint_parameters': {
        'column_names': ['Category', 'Product']
    }
}

constraint_1 = {
    'constraint_class': 'FixedCombinations',
    'constraint_parameters': {
        'column_names': ['Region', 'City']
    }
}

# metadata.update_column(
#     column_name='ID',
#     sdtype='id',
#     regex_format='ID[0-9]{5}')

# metadata.set_primary_key(column_name='ID')

### GaussianCopulaSynthesizer

In [65]:
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer = GaussianCopulaSynthesizer(metadata, default_distribution = 'uniform')
synthesizer.add_constraints(constraints=[
    constraint
])
synthesizer.add_constraints(constraints=[
    constraint_1
])
synthesizer.fit(df_final_mean)
synthetic_data = synthesizer.sample(num_rows=1000)

Sampling rows: 100%|████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 16612.55it/s]


In [66]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=df_final,
    synthetic_data=synthetic_data,
    metadata=metadata)

C:\Users\shashi.kiran\Anaconda3\lib\site-packages\xarray\core\pycompat.py:22: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion(duck_array_module.__version__)
C:\Users\shashi.kiran\Anaconda3\lib\site-packages\xarray\core\pycompat.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  duck_array_version = LooseVersion("0.0.0")
C:\Users\shashi.kiran\Anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\shashi.kiran\Anaconda3\lib\site-packages\xarray\core\npcompat.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) >= "1.20.0":
C:\Users\shashi.kiran\Anaconda3\lib\site-packages\xarray\core\pdcompat.py:45: DeprecationWarning: distutils Version c


Overall Quality Score: 40.82%

Properties:
Column Shapes: 64.39%
Column Pair Trends: 17.25%


In [67]:
# synthetic_data.to_csv('processed_with_append.csv')

### CTGANSynthesizer

In [73]:
from sdv.single_table import CTGANSynthesizer
synthesizer = CTGANSynthesizer(metadata)
synthesizer.add_constraints(constraints=[
    constraint
])
synthesizer.add_constraints(constraints=[
    constraint_1
])
synthesizer.fit(df_final_mean)
synthetic_data = synthesizer.sample(num_rows=1000)

Sampling rows: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3935.51it/s]


In [74]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=df_final,
    synthetic_data=synthetic_data,
    metadata=metadata)

Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.19it/s]



Overall Quality Score: 42.48%

Properties:
Column Shapes: 62.67%
Column Pair Trends: 22.3%


### CopulaGANSynthesizer

In [75]:
from sdv.single_table import CopulaGANSynthesizer
synthesizer = CopulaGANSynthesizer(metadata)
synthesizer.add_constraints(constraints=[
    constraint
])
synthesizer.add_constraints(constraints=[
    constraint_1
])
synthesizer.fit(df_final_mean)
synthetic_data = synthesizer.sample(num_rows=1000)

Sampling rows: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3752.55it/s]


In [76]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=df_final,
    synthetic_data=synthetic_data,
    metadata=metadata)

Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.41it/s]



Overall Quality Score: 41.62%

Properties:
Column Shapes: 61.96%
Column Pair Trends: 21.29%


### TVAESynthesizer

In [77]:
from sdv.single_table import TVAESynthesizer
synthesizer = TVAESynthesizer(metadata)
synthesizer.add_constraints(constraints=[
    constraint
])
synthesizer.add_constraints(constraints=[
    constraint_1
])
synthesizer.fit(df_final_mean)
synthetic_data = synthesizer.sample(num_rows=1000)

Sampling rows: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 9862.17it/s]


In [78]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=df_final,
    synthetic_data=synthetic_data,
    metadata=metadata)

Creating report:   0%|                                                                           | 0/4 [00:00<?, ?it/s]C:\Users\shashi.kiran\Anaconda3\lib\site-packages\sdmetrics\column_pairs\statistical\correlation_similarity.py:78: ConstantInputWarning: The synthetic data in column 'UnitPrice' contains a constant value. Correlation is undefined for constant data.
  warnings.warn(ConstantInputWarning(msg))
C:\Users\shashi.kiran\Anaconda3\lib\site-packages\sdmetrics\column_pairs\statistical\correlation_similarity.py:78: ConstantInputWarning: The synthetic data in column 'UnitPrice' contains a constant value. Correlation is undefined for constant data.
  warnings.warn(ConstantInputWarning(msg))
Creating report: 100%|███████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.27it/s]



Overall Quality Score: 19.76%

Properties:
Column Shapes: 31.57%
Column Pair Trends: 7.95%


### Gretal AI